# Task description

Introduction
At its core, churn prediction is a classification problem, where the classes often are ‘churned’ and ‘active’. The prediction is based on historical data, including customer behavior, demographics, transaction history, and more.

Goal: well-balanced classification model.

Instructions:
Work in Python or R.
Examine the data.
While working through the layers of challenge, please leave comments in your code.
Share your solution in a notebook format with us.



# Task execution

In [3]:
import pandas as pd
import numpy as np

# Loading and displaying dataset

In [4]:
churn_df = pd.read_csv("../data/task_data_churned.csv")

In [5]:
pd.set_option('display.max_columns', None)

# DataFrame structure introduction

In [6]:
churn_df.columns

Index(['ws_users_activated', 'ws_users_deactivated', 'ws_users_invited',
       'action_create_project', 'action_export_report',
       'action_api_and_webhooks', 'action_time_entries_via_tracker',
       'action_start_trial', 'action_import_csv', 'action_create_invoice',
       'action_lock_entries', 'action_add_targets',
       'action_connect_quickbooks', 'action_create_expense',
       'action_project_budget', 'action_gps_tracking', 'action_screenshots',
       'action_create_custom_field', 'country', 'value_days_to_purchase',
       'value_number_of_active_months', 'value_transactions_number',
       'value_regular_seats', 'value_kiosk_seats', 'revenue',
       'churned_status'],
      dtype='object')

In [7]:
churn_df.dtypes

ws_users_activated                   int64
ws_users_deactivated                 int64
ws_users_invited                     int64
action_create_project                int64
action_export_report                 int64
action_api_and_webhooks              int64
action_time_entries_via_tracker      int64
action_start_trial                   int64
action_import_csv                    int64
action_create_invoice                int64
action_lock_entries                  int64
action_add_targets                   int64
action_connect_quickbooks            int64
action_create_expense                int64
action_project_budget                int64
action_gps_tracking                float64
action_screenshots                 float64
action_create_custom_field         float64
country                             object
value_days_to_purchase               int64
value_number_of_active_months        int64
value_transactions_number            int64
value_regular_seats                  int64
value_kiosk

In [8]:
churn_df.head()

,ws_users_activated,ws_users_deactivated,ws_users_invited,action_create_project,action_export_report,action_api_and_webhooks,action_time_entries_via_tracker,action_start_trial,action_import_csv,action_create_invoice,action_lock_entries,action_add_targets,action_connect_quickbooks,action_create_expense,action_project_budget,action_gps_tracking,action_screenshots,action_create_custom_field,country,value_days_to_purchase,value_number_of_active_months,value_transactions_number,value_regular_seats,value_kiosk_seats,revenue,churned_status
0,3,2,0,5,8,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,Canada,2,0,6,3,0,184.925,No
1,6,1,0,35,106,0,33,0,1,0,5,8,0,0,3,NaN,NaN,3.0,United Kingdom,37,9,9,6,0,608.842,No
2,2,0,0,3,3,0,0,0,0,0,10,2,1,0,9,NaN,1.0,NaN,Florida,98,3,12,3,0,395.122,No
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,1.0,NaN,Kentucky,0,1,2,1,0,25.974,Yes
4,1,0,0,0,0,1,0,0,0,0,0,0,2,0,0,NaN,NaN,NaN,Ireland,21,2,3,1,0,38.961,Yes


## Null values handling 

In [9]:
missing_values = churn_df.isnull().sum()
print(missing_values)

ws_users_activated                    0
ws_users_deactivated                  0
ws_users_invited                      0
action_create_project                 0
action_export_report                  0
action_api_and_webhooks               0
action_time_entries_via_tracker       0
action_start_trial                    0
action_import_csv                     0
action_create_invoice                 0
action_lock_entries                   0
action_add_targets                    0
action_connect_quickbooks             0
action_create_expense                 0
action_project_budget                 0
action_gps_tracking                1626
action_screenshots                 1458
action_create_custom_field         2059
country                              84
value_days_to_purchase                0
value_number_of_active_months         0
value_transactions_number             0
value_regular_seats                   0
value_kiosk_seats                     0
revenue                               0


As we can see here, a lot of people didn't want their GPS to be tracked, and they didn't add screenshots. So we can discard those columns entirely as they are empty on more than 50% of the ocassions. Also, we could exclude records that don't have a country, because they make a really small sample.

Removing action_screenshot might be two-edged sword, because someone that has a lot of these screenshots taken pays a lot of attention to that. To rephrase - if company measures their employees productivity using screenshots of their screen means that they are highly unlikely to opt out of the product. So we will fill null values with zeros.

In [10]:
columns_to_exclude = ['action_gps_tracking',  'action_create_custom_field']
churn_df = churn_df.drop(columns=columns_to_exclude)

In [11]:
churn_df['action_screenshots'] = churn_df['action_screenshots'].fillna(0)

In [12]:
churn_df = churn_df[churn_df['country'].notnull()]

In [13]:
missing_values = churn_df.isnull().sum()
print(missing_values)

ws_users_activated                 0
ws_users_deactivated               0
ws_users_invited                   0
action_create_project              0
action_export_report               0
action_api_and_webhooks            0
action_time_entries_via_tracker    0
action_start_trial                 0
action_import_csv                  0
action_create_invoice              0
action_lock_entries                0
action_add_targets                 0
action_connect_quickbooks          0
action_create_expense              0
action_project_budget              0
action_screenshots                 0
country                            0
value_days_to_purchase             0
value_number_of_active_months      0
value_transactions_number          0
value_regular_seats                0
value_kiosk_seats                  0
revenue                            0
churned_status                     0
dtype: int64


In [14]:
churn_df.dtypes

ws_users_activated                   int64
ws_users_deactivated                 int64
ws_users_invited                     int64
action_create_project                int64
action_export_report                 int64
action_api_and_webhooks              int64
action_time_entries_via_tracker      int64
action_start_trial                   int64
action_import_csv                    int64
action_create_invoice                int64
action_lock_entries                  int64
action_add_targets                   int64
action_connect_quickbooks            int64
action_create_expense                int64
action_project_budget                int64
action_screenshots                 float64
country                             object
value_days_to_purchase               int64
value_number_of_active_months        int64
value_transactions_number            int64
value_regular_seats                  int64
value_kiosk_seats                    int64
revenue                            float64
churned_sta

Now the data doesn't have any missing values. 

Every column is now numeric, except the country and churned status. 

# Outliers handling

In [15]:
churn_df.describe()

,ws_users_activated,ws_users_deactivated,ws_users_invited,action_create_project,action_export_report,action_api_and_webhooks,action_time_entries_via_tracker,action_start_trial,action_import_csv,action_create_invoice,action_lock_entries,action_add_targets,action_connect_quickbooks,action_create_expense,action_project_budget,action_screenshots,value_days_to_purchase,value_number_of_active_months,value_transactions_number,value_regular_seats,value_kiosk_seats,revenue
count,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000,2418.000000
mean,5.198925,0.742763,0.144748,26.585194,20.436725,0.345740,18.315964,0.173284,0.573201,8.495451,1.541770,0.285773,0.075269,9.650538,10.193962,0.581059,59.259305,4.134822,5.607940,5.612903,0.256824,341.362419
std,9.555736,3.275768,0.662657,73.927866,68.397778,2.898572,112.463586,0.378571,4.336085,53.449130,6.534789,1.330015,0.652262,73.086291,37.067629,0.851539,83.369941,3.650429,4.757518,9.907175,2.975803,809.238613
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,1.000000,0.000000,38.961000
50%,2.000000,0.000000,0.000000,8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,3.000000,4.000000,2.000000,0.000000,103.896000
75%,6.000000,0.000000,0.000000,25.000000,14.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,1.000000,79.000000,7.000000,8.000000,6.000000,0.000000,311.688000
max,158.000000,73.000000,17.000000,1923.000000,1431.000000,127.000000,3382.000000,1.000000,120.000000,1405.000000,139.000000,30.000000,27.000000,1785.000000,829.000000,11.000000,420.000000,14.000000,90.000000,170.000000,117.000000,20844.252000


Looking at this table, we can see that a lot of data has been skewed by the maximum values (probably large clients). Let's take for example `action_api_and_webhooks`. Mean of that column is 0.38 and std is around 3.086. But the maximum value is 127, which skews picture heavily.

Before we jump into solving this issue, we have to see that all of the numbers are non-negative. So we will trim down all of the top 3% for each of the column. This won't be done iteratively, because that will lead to having $(0.97)^{num\_ of\_ columns}$ which would lead to having very small sample of data to train the model on (especially in our case where we have a lot of columns). So we will create a function that tests this for all of the numeric columns.

In [16]:
def calculate_quantiles(dataframe:pd.DataFrame) -> dict:
    """
    Calculate the 97% quantile for each numeric column in a Pandas DataFrame.

    Parameters:
    dataframe (pd.DataFrame): The input DataFrame.

    Returns:
    (dict): A dictionary where keys are numeric column names, and values are the 97% quantiles.
    """
    quantiles_dict = {}
    
    # Select only numeric columns
    numeric_columns = dataframe.select_dtypes(include=[np.number])
    
    # Loop through each numeric column
    for column in numeric_columns.columns:
        # Calculate the 98% quantile for the column
        quantile_97 = numeric_columns[column].quantile(0.97)
        # Add the result to the dictionary
        quantiles_dict[column] = quantile_97
    
    return quantiles_dict

In [17]:
quantile_97th = calculate_quantiles(churn_df)

In [18]:
def is_record_within_quantiles(record: pd.Series) -> bool:
    """
    Check if a single record (row) falls within the quantile values specified in a dictionary.

    Parameters:
    record (pd.Series): A single row (record) from a DataFrame.
    quantile_dict (dict): A dictionary where keys are column names, and values are the quantile values.

    Returns:
    (bool): True if the record falls within the quantiles for all columns, False otherwise.
    """
    # Initialize a variable to track whether the record is within quantiles
    is_within_quantiles = True

    # Loop through each column in the record
    for column in quantile_97th.keys():
        value = record[column]
        # Check if the record value is less than or equal to the quantile value for the column
        if value > quantile_97th[column]:
            is_within_quantiles = False
            break  # Exit the loop early if a column is not within quantiles

    return is_within_quantiles

In [19]:
churn_df['isWithinQuantile'] = churn_df.apply(lambda row: is_record_within_quantiles(row), axis=1)


In [20]:
churn_df['isWithinQuantile'].value_counts()

isWithinQuantile
True     1715
False     703
Name: count, dtype: int64

In [21]:
no_outlier_churf_df = churn_df[churn_df['isWithinQuantile'] == True]
no_outlier_churf_df.drop(['isWithinQuantile'], inplace=True, axis=1)

C:\Users\DT User\AppData\Local\Temp\ipykernel_14744\2419240485.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  no_outlier_churf_df.drop(['isWithinQuantile'], inplace=True, axis=1)


In [22]:
no_outlier_churf_df.describe()

,ws_users_activated,ws_users_deactivated,ws_users_invited,action_create_project,action_export_report,action_api_and_webhooks,action_time_entries_via_tracker,action_start_trial,action_import_csv,action_create_invoice,action_lock_entries,action_add_targets,action_connect_quickbooks,action_create_expense,action_project_budget,action_screenshots,value_days_to_purchase,value_number_of_active_months,value_transactions_number,value_regular_seats,value_kiosk_seats,revenue
count,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000,1715.000000
mean,2.872303,0.234985,0.061224,13.135277,7.480466,0.104373,3.736443,0.137026,0.062974,2.837318,0.523032,0.087464,0.027988,1.622741,3.727697,0.493294,40.450146,3.539359,4.497959,3.183673,0.018659,156.692108
std,3.159604,0.720714,0.239812,20.642490,17.073014,0.340153,15.034594,0.343975,0.378228,7.406343,1.707257,0.342345,0.164987,6.588723,10.472719,0.658254,60.638649,3.245326,3.350120,3.421493,0.135357,222.193438
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.000000,1.000000,0.000000,25.974000
50%,2.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.000000,3.000000,4.000000,2.000000,0.000000,77.909000
75%,4.000000,0.000000,0.000000,16.000000,6.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,50.000000,6.000000,6.000000,4.000000,0.000000,173.719000
max,24.000000,5.000000,1.000000,147.000000,129.000000,2.000000,138.000000,1.000000,4.000000,51.000000,13.000000,2.000000,1.000000,65.000000,82.000000,2.000000,285.000000,12.000000,16.000000,24.000000,1.000000,1793.857000


In [23]:
no_outlier_churf_df['churned_status'].value_counts()

churned_status
No     1081
Yes     634
Name: count, dtype: int64

In [25]:
no_outlier_churf_df.to_csv("../data/task_data_churned_clean.csv", index=False)

As we can see here, results look mode natural. Of course, we wouldn't exclude a lot of those, because we would lose a general pattern. But those ones that heavily were skewing the picture are now removed.

Another way at tackling this would be taking interval $(mean - 3*std, mean + 3*std)$ which is also pretty common. But due to lack of time and testing capabilties we have chosen approach shown above.